In [24]:
import os
import scipy as sp
import rsmf
import matplotlib
from matplotlib import pyplot as plt
from postProcessing.importData import *


#Insert name here
NAME = "64cl_u_s3" 
# first type 64cl_u_s1
# second type 64cl_u_s3
#################


PATH = "./simulations/" + NAME
if(os.path.isdir(NAME)):
    print("Enter a valid file name")
else:
    PATH += ".h5"
    data_0 = dataProcessing(PATH)


b'float'
SELF CONSISTENT [GAUGE, HARTREE]  ==> [True, False]
The current simulation does not export HoppingFields
Note: Magnetic free energy density not added to free energy density because defined on plaquettes


In [25]:
NAME = "64cl_v_s3" #11-vortex case 6422JO-1.70c12-1.h5 ; 7-vortex case #4822W-1, 11-vortex case 6422JO-1.70c19-1.h5 ; 4822J1, 4822J, 4822JL-1, 4822JL-10, 64MJO-1.70c19.h5
# first type - 64cl_v, 64cl_11, 64_cl_18, 64cl_LV, 64cl_LV18
# second type - 64cl_v_s3, 64cl_s_18, 64cl_s_11, 64cl_s_7, 64cl_s_3, 64cl_s_3-1, 64cl_s_3-1-1
# third type - 64cl_sv_2-2, 64cl_sv_18, 64cl_sv_11, 64cl_sv_7, 64cl_sv_LV_11, 64cl_sv_LV_5
# fourth type 64cl_sv_4
PATH = "./simulations/" + NAME
if(os.path.isdir(NAME)):
    print("Enter a valid file name")
else:
    PATH += ".h5"
    data = dataProcessing(PATH)

b'float'
SELF CONSISTENT [GAUGE, HARTREE]  ==> [True, False]
The current simulation does not export HoppingFields
Note: Magnetic free energy density not added to free energy density because defined on plaquettes


In [58]:
fmt= rsmf.setup(r"\documentclass[aps, prl, superscriptaddress, twocolumn, letterpaper, 10pt, amsfonts, amsmath, amssymb, groupedaddress, longbibliography]{revtex4-2}")

In [59]:
edge = 2
cp = 37
row = 32
d_1 = (np.abs(data_0.D_1) - np.abs(data.D_1))[row, cp:-edge]
d_2 = (np.abs(data_0.D_2) - np.abs(data.D_2))[row, cp:-edge]
f_b = (np.abs(data.B))[row, cp:-edge+1]

In [60]:
def fit_f(x, a1, c1, a2, c2):
    return a1 * np.exp(-c1 * x) + a2 * np.exp(-c2 * x)

In [61]:
def cost_function(args, x1, y1, x2, y2):
    a1, a2, b1, b2, c1, c2 = args
    res_y_1 = a1 * np.exp(-c1 * x1) + a2 * np.exp(-c2 * x1)
    res_y_2 = b1 * np.exp(-c1 * x2) + b2 * np.exp(-c2 * x2)
    return np.sum((res_y_1 - y1) ** 2 / y1 ** 2) + np.sum((res_y_2 - y2) ** 2 / y2 ** 2)

In [62]:
def fit_B(x, a1, c1):
    return a1 * np.exp(-c1 * x)

In [63]:
x = np.linspace((37-32) / 64, 0.5, len(d_1))

In [64]:
opt = sp.optimize.minimize(cost_function, (0.02, 0.3, 0.1, 0.07, 12.2, 20.3), args=(x, d_1, x, d_2))

In [65]:
opt_B = sp.optimize.curve_fit(fit_B, x, f_b)

In [66]:
def sum_e(x1, a1, a2, c1, c2):
    return a1 * np.exp(-c1 * x1) + a2 * np.exp(-c2 * x1)

In [67]:
a1, a2, b1, b2, c1, c2 = opt.x

In [68]:
64 / c1, 64 / c2

(4.983028480985676, 1.7356066406263078)

In [69]:
a1, a2, b1, b2

(0.0010290987262907866,
 0.3589832644378638,
 0.02887019791698209,
 0.13113302242234867)

In [70]:
64/opt_B[0][1] * np.sqrt(2)

3.298466678733822

In [71]:
opt_B[0][-1]

27.439921881103228

In [72]:
start = 0
fin = -1

In [73]:
fig = fmt.figure()
bl = plt.rcParams['axes.linewidth']
gs = fig.add_gridspec(3,2,width_ratios=[1.3,1])
ax1= fig.add_subplot(gs[:, 0])
ax3= fig.add_subplot(gs[0, 1])
ax4= fig.add_subplot(gs[1, 1])
ax5= fig.add_subplot(gs[2, 1])

ax=plt.gca()
color = next(ax._get_lines.prop_cycler)['color']
ax1.set_xlabel(r'$r/L$')
ax1.set_ylabel(r'$\left| \Delta \left(\infty\right) \right| - \left| \Delta \left(r\right) \right|$')
ax1.set_yscale('log')

cmap = plt.get_cmap('Oranges')
color = cmap(0.75)
ax1.scatter(x[start:fin], d_1[start:fin], label = r'$\Delta_1$', color=color)
ax1.plot(x[start:fin], sum_e(x[start:fin], a1, a2, c1, c2), color=color)

cmap = plt.get_cmap('Greens')
color = cmap(0.75)
ax1.scatter(x[start:fin], d_2[start:fin], label = r'$\Delta_2$', color=color)
ax1.plot(x[start:fin], sum_e(x[start:fin], b1, b2, c1, c2), color=color)
ax1.tick_params(axis='y')

cmap = plt.get_cmap('Purples')
color = cmap(0.75)
ax2 = ax1.twinx()  
ax2.set_ylabel(r'$\sqrt{B}$')  
ax2.scatter(x[start:fin], f_b[start:fin] ** 0.5, label = r'$\sqrt{B}$', color=color)
ax2.plot(x[start:fin], fit_B(x[start:fin], *opt_B[0]) ** 0.5, color=color)
ax2.set_yscale('log')

plt.set_cmap('Oranges')
ax3.imshow(np.abs(data.D_1))
ax3.axhline(y = 32, xmin = cp / 64, xmax = 1, linestyle = ':', color='black')
ax3.text(0.0, 1.0, r'$\left| \Delta_1 \right|$', verticalalignment='top', bbox=dict(facecolor='1', edgecolor='0', pad=3.0, linewidth=bl))
cbar = fig.colorbar(ax3.imshow(np.abs(data.D_1)), ax=ax3, shrink=1.0, aspect=10)
cbar.ax.tick_params(direction='out', length=1.5)

plt.set_cmap('Greens')
ax4.imshow(np.abs(data.D_2))
ax4.axhline(y = 32, xmin = cp / 64, xmax = 1, linestyle = ':', color='black')
ax4.text(0.0, 1.0, r'$\left| \Delta_2 \right|$', verticalalignment='top', bbox=dict(facecolor='1', edgecolor='0', pad=3.0,  linewidth=bl))
cbar = fig.colorbar(ax4.imshow(np.abs(data.D_2)), ax=ax4, shrink=1.0, aspect=10)
cbar.ax.tick_params(direction='out', length=1.5)

plt.set_cmap('Purples')
ax5.imshow(data.B)
ax5.axhline(y = 32, xmin = cp / 64, xmax = 1, linestyle = ':', color='black')
ax5.text(0.0, 1.0, r'$B$', verticalalignment='top', bbox=dict(facecolor='1', edgecolor='0', pad=3.0, linewidth=bl))
cbar = fig.colorbar(ax5.imshow(data.B), ax=ax5, shrink=1.0, aspect=10)
cbar.ax.tick_params(direction='out', length=1.5)

ax3.xaxis.set_visible(False)
ax3.yaxis.set_visible(False)
ax4.xaxis.set_visible(False)
ax4.yaxis.set_visible(False)
ax5.yaxis.set_visible(False)

fig.tight_layout() 
ax1.legend(loc=3)
ax2.legend(loc=1)

plt.subplots_adjust(hspace=0.05)
plt.savefig('images/single_vortex.pdf')
plt.show()

/tmp/ipykernel_11467/1960947747.py:67: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()
